In [ ]:
!conda install -c conda-forge pytorch=2.2.0 torchvision=0.17.0 torchtext=0.17.2 cpuonly numpy pandas matplotlib scikit-learn tqdm -y

In [ ]:
!pip install datasets==2.20.0 trl==0.9.6 transformers==4.42.3 peft==0.11.1 sacrebleu==2.4.2 evaluate==0.4.2

 # Preference Tuning with DPO

When we use the Hugging Face stack, preference tuning is eerily similar to
the instruction tuning we covered before with some slight differences. We
will still be using TinyLlama but this time an instruction-tuned version that
was first trained using full fine-tuning and then further aligned with DPO.
Compared to our initial instruction-tuned model, this LLM was trained on
much larger datasets.
In this section, we will demonstrate how you can further align this model
using DPO with reward-based datasets.

In [2]:
from datasets import load_dataset


def format_prompt(example):
    """Format the prompt to using the <|user|> template TinyLLama
    is using"""
    # Format answers
    system = "<|system|>\n" + example["system"] + "</s>\n"
    prompt = "<|user|>\n" + example["input"] + " </s>\n<|assistant|>\n"
    chosen = example["chosen"] + "</s>\n"
    rejected = example["rejected"] + "</s>\n"
    return {
        "prompt": system + prompt,
        "chosen": chosen,
        "rejected": rejected,
    }


# Apply formatting to the dataset and select relatively short answers
dpo_dataset = load_dataset("argilla/distilabel-intel-orca-dpo-pairs", split="train")
dpo_dataset = dpo_dataset.filter(
    lambda r: 
        r["status"] != "tie"
        and r["chosen_score"] >= 8
        and not r["in_gsm8k_train"]
)
dpo_dataset = dpo_dataset.map(format_prompt, remove_columns=dpo_dataset.column_names)
dpo_dataset

/home/pedro/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset({
    features: ['chosen', 'rejected', 'prompt'],
    num_rows: 5922
})

Note that we apply additional filtering to further reduce the size of the data
to roughly 6,000 examples from the original 13,000 examples.

# Model Quantization

We load our base model and load it with the LoRA we created previously.
As before, we quantize the model to reduce the necessary VRAM for
training:

In [ ]:
from transformers import (
    BitsAndBytesConfig,
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
)

# 4-bit quantization configuration - Q in QLoRA
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Use 4-bit precision model loading
    bnb_4bit_quant_type="nf4",  # Quantization type
    bnb_4bit_compute_dtype="float16",  # Compute dtype
    bnb_4bit_use_double_quant=True,  # Apply nested quantization
)

# Load LLaMA tokenizer
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = "<PAD>"
tokenizer.padding_side = "left"

# Merge LoRA and base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Next, we use the same LoRA configuration as before to perform the DPO
training:

In [ ]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

# Prepare LoRA configuration
peft_config = LoraConfig(
    lora_alpha=32,  # LoRA Scaling
    lora_dropout=0.05,  # Dropout for LoRA Layers
    r=64,  # Rank
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[  # Layers to target
        "k_proj",
        "gate_proj",
        "v_proj",
        "up_proj",
        "q_proj",
        "o_proj",
        "down_proj",
    ],
)
# prepare model for training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

# Training Configuration

For the sake of simplicity, we will use the same training arguments as we
did before with one difference. Instead of running for a single epoch (which
can take up to two hours), we run for 200 steps instead for illustration
purposes. Moreover, we added the warmup_ratio parameter, which
increases the learning rate from 0 to the learning_rate value we set for
the first 10% of steps. By maintaining a small learning rate at the start (i.e.,warmup period), we allow the model to adjust to the data before applying
larger learning rates, therefore avoiding harmful divergence:

In [ ]:
from trl import DPOConfig

output_dir = "./dpo-tinyllama"

# Training arguments
training_arguments = DPOConfig(
    output_dir=output_dir,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    learning_rate=1e-5,
    lr_scheduler_type="cosine",
    max_steps=200,
    logging_steps=10,
    fp16=True,
    gradient_checkpointing=True,
    warmup_ratio=0.1,
    beta=0.1,
    label_smoothing=0.0,
    loss_type="sigmoid",
)

# Training

Now that we have prepared all our models and parameters, we can start
fine-tuning our model:

In [ ]:
from trl import DPOTrainer

# Create DPO trainer
dpo_trainer = DPOTrainer(
    model, 
    ref_model=None, # The reference model (not used in this case because LoRA has been used)
    args=training_arguments,
    train_dataset=dpo_dataset,
    tokenizer=tokenizer,
    peft_config=peft_config,
    beta=0.1,
    max_prompt_length=512,
    max_length=512
)

/home/pedro/anaconda3/envs/torch-env/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_prompt_length, max_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in DPOTrainer, please use the DPOConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/home/pedro/anaconda3/envs/torch-env/lib/python3.10/site-packages/peft/tuners/lora/bnb.py:325: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(
/home/pedro/.local/lib/python3.10/site-packages/trl/trainer/dpo_trainer.py:675: UserWarning: You passed `max_length` to the DPOTrainer, the value you passed will override the one in the `DPOConfig`.
  warnings.warn(
/home/pedro/.local/lib/python3.10/site-packages/trl/trainer/dpo_trainer.py:688: UserWarning: You passed `max_prompt_length` to the DPOTrainer, the value you passed will override

In [7]:
# Start fine tuning 
dpo_trainer.train()

  0%|          | 0/200 [00:00<?, ?it/s]/home/pedro/anaconda3/envs/torch-env/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/pedro/anaconda3/envs/torch-env/lib/python3.10/site-packages/torch/utils/checkpoint.py:90: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
Could not estimate the number of tokens of the input, floating-point operations will not be computed
  5%|▌         | 10/200 [01:05<20:09,  6.36s/it]

{'loss': 0.6913, 'grad_norm': 2.3933210372924805, 'learning_rate': 5e-06, 'rewards/chosen': 0.00039761661901138723, 'rewards/rejected': -0.00327131524682045, 'rewards/accuracies': 0.30000001192092896, 'rewards/margins': 0.0036689317785203457, 'logps/rejected': -91.34730529785156, 'logps/chosen': -83.61310577392578, 'logits/rejected': -3.0855438709259033, 'logits/chosen': -3.0500998497009277, 'epoch': 0.01}


 10%|█         | 20/200 [02:10<19:56,  6.64s/it]

{'loss': 0.673, 'grad_norm': 2.6381709575653076, 'learning_rate': 1e-05, 'rewards/chosen': -0.003814463736489415, 'rewards/rejected': -0.046028126031160355, 'rewards/accuracies': 0.4625000059604645, 'rewards/margins': 0.04221365973353386, 'logps/rejected': -130.34994506835938, 'logps/chosen': -98.92628479003906, 'logits/rejected': -3.1642000675201416, 'logits/chosen': -3.0853583812713623, 'epoch': 0.03}


 15%|█▌        | 30/200 [03:18<19:03,  6.73s/it]

...

100%|██████████| 200/200 [21:42<00:00,  6.12s/it]

{'loss': 0.5317, 'grad_norm': 3.9176714420318604, 'learning_rate': 7.615242180436521e-10, 'rewards/chosen': -0.16919878125190735, 'rewards/rejected': -1.207228183746338, 'rewards/accuracies': 0.4625000059604645, 'rewards/margins': 1.0380291938781738, 'logps/rejected': -134.67039489746094, 'logps/chosen': -79.00543975830078, 'logits/rejected': -3.1448285579681396, 'logits/chosen': -3.0928685665130615, 'epoch': 0.27}


100%|██████████| 200/200 [21:45<00:00,  6.53s/it]

{'train_runtime': 1305.2724, 'train_samples_per_second': 1.226, 'train_steps_per_second': 0.153, 'train_loss': 0.5910451936721802, 'epoch': 0.27}


TrainOutput(global_step=200, training_loss=0.5910451936721802, metrics={'train_runtime': 1305.2724, 'train_samples_per_second': 1.226, 'train_steps_per_second': 0.153, 'total_flos': 0.0, 'train_loss': 0.5910451936721802, 'epoch': 0.2701789935832489})

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

log = pd.DataFrame(dpo_trainer.state.log_history)
log_t = log[log["loss"].notna()]
log_e = log[log["eval_loss"].notna()]

# Plot train and evaluation loss
plt.plot(log_t["epoch"], log_t["loss"], label="Train")
plt.plot(log_e["epoch"], log_e["eval_loss"], label="Eval")
plt.title("Model Losses")
plt.legend()
plt.show()

### Notes

Using `save_model()` we save the LoRA adapters alongside the models. This means that `output_dir` will contain both the model weights and LoRA adapters as different modules. 

The function `merge_and_unload()` create a standalone model by merging the model weights with the LoRA adapters. This step also free precious VRAM by unloading the LoRA adapters which are no longer necessary.

In [ ]:
# Save the adapter 
dpo_trainer.save_model(output_dir)

# Merge the base model weights to LoRA adapters
# Also free vram by unloading LoRA adapters 
merged_model = model.merge_and_unload()

In [ ]:
"""from peft import PeftModel

# Load the model with the LoRA adapters active
dpo_model = PeftModel.from_pretrained(
    model,
    output_dir,
    device_map="auto",
)

merged_model = dpo_model.merge_and_unload()"""

/home/pedro/anaconda3/envs/torch-env/lib/python3.10/site-packages/peft/tuners/lora/bnb.py:325: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [ ]:
# Save model and tokenizer
merged_dir = "./dpo-tinyllama-merged"

merged_model.save_pretrained(merged_dir)
tokenizer.save_pretrained(merged_dir)

('./dpo-tinyllama-merged/tokenizer_config.json',
 './dpo-tinyllama-merged/special_tokens_map.json',
 './dpo-tinyllama-merged/tokenizer.json')

# Testing inference

Let's load our fine-tuned model as we would do in a production pipeline.

In [ ]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    pipeline,
)

model_dir = "./dpo-tinyllama-merged"

finetuned_model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    device_map="auto",
    trust_remote_code=True,
)

finetuned_tokenizer = AutoTokenizer.from_pretrained(
    model_dir,
    trust_remote_code=True
)

pipe = pipeline(
    "text-generation",
    model=finetuned_model,
    tokenizer=finetuned_tokenizer,
    max_new_tokens=256,
    temperature=0.7,
    do_sample=True,
)

prompt = "Explain quantum computing in simple terms."
print(pipe(prompt)[0]["generated_text"])

Explain quantum computing in simple terms. Let's say you have to find the square roots of numbers. In classical computing, you could use a calculator to do this, but quantum computers can solve this problem in a fraction of the time. How does quantum computing work?

A: Quantum computing works by using quantum bits, or qubits, which can take on both a 0 or a 1, and can exist in multiple states at the same time. A quantum computer can solve problems that are too computationally intensive for classical computers by using a technique called quantum entanglement.

When two qubits are entangled, they can interact with each other in ways that are not possible for classical bits. If two qubits are entangled, they can perform a quantum computation by acting on one qubit simultaneously with the other qubit. This means that if one qubit performs a calculation, the other qubit can act on it simultaneously, either with a 0 or with a 1, and the result will be the same.

In classical computers, the 